# Evaluating job posts
This notebook provides sample code for conducting a text analysis using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and other research with AI agents and large language models. 

Using a dataset of job posts as an example, we demonstrate how to: 

1. Import data into EDSL 
2. Create questions about the data 
3. Design an AI agent to answer the questions
4. Select a language model to generate responses
5. Analyze results as a formatted dataset


## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install EDSL](https://docs.expectedparrot.com/en/latest/installation.html).
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) to run surveys at the Expected Parrot server. You can use an Expected Parrot API key or your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) also provides examples of EDSL basic components. 

## Selecting data for review
First we identify some data for review. Data can be created using the EDSL tools or [imported from other sources](https://docs.expectedparrot.com/en/latest/scenarios.html). For purposes of this demo we import a set of job posts:

In [1]:
job_posts = [
    "Oversee daily operations, manage staff, and ensure customer satisfaction in a fast-paced retail environment.",
    "Craft engaging and informative blog posts on health and wellness topics to boost website traffic and engage readers.",
    "Analyze sales data using statistical tools to identify trends and provide actionable insights to the marketing team.",
    "Prepare gourmet dishes that comply with restaurant standards and delight customers with unique flavor combinations.",
    "Design creative visual content for marketing materials, including brochures, banners, and digital ads, using Adobe Creative Suite.",
    "Develop, test, and maintain robust software solutions to improve business processes using Python and Java.",
    "Craft coffee drinks and manage the coffee station while providing excellent customer service in a busy café.",
    "Manage recruitment processes, conduct interviews, and oversee employee benefit programs to ensure a motivated workforce.",
    "Assist veterinarians by preparing animals for surgery, administering injections, and providing post-operative care.",
    "Design aesthetic and practical outdoor spaces for clients, from residential gardens to public parks.",
    "Install and repair residential plumbing systems, including water heaters, pipes, and fixtures to ensure proper functionality.",
    "Develop comprehensive marketing strategies that align with company goals, including digital campaigns and branding efforts.",
    "Install, maintain, and repair electrical wiring, equipment, and fixtures to ensure safe and effective operation.",
    "Provide personalized fitness programs and conduct group fitness classes to help clients achieve their health goals.",
    "Diagnose and repair automotive issues, perform routine maintenance, and ensure vehicles meet safety standards.",
    "Lead creative campaigns, from concept through execution, coordinating with graphic designers and content creators.",
    "Educate students in mathematics using innovative teaching strategies to enhance understanding and interest in the subject.",
    "Drive sales through engaging customer interactions, understanding client needs, and providing product solutions.",
    "Fold dough into pretzel shapes ensuring each is uniformly twisted and perfectly salted before baking.",
    "Address customer inquiries and issues via phone and email, ensuring high levels of satisfaction and timely resolution.",
]

## Constructing questions about the data
Next we create some questions about the data. EDSL provides a variety of question types that we can choose from based on the form of the response that we want to get back from the model (multiple choice, free text, checkbox, linear scale, etc.). [Learn more about question types](https://docs.expectedparrot.com/en/latest/questions.html). 

Note that we use a `{{ placeholder }}` in each question text in order to parameterize the questions with the individual job posts in the next step:

In [2]:
from edsl import (
    QuestionList,
    QuestionLinearScale,
    QuestionMultipleChoice,
    QuestionYesNo,
    QuestionFreeText,
)

q1 = QuestionList(
    question_name="category_list",
    question_text="Draft a list of increasingly specific categories for the following job post: {{ job_post }}",
    max_list_items=3,  # optional
)

q2 = QuestionLinearScale(
    question_name="specific_scale",
    question_text="How specific is this job post: {{ job_post }}",
    question_options=[0, 1, 2, 3, 4, 5],
    option_labels={0: "Unclear", 1: "Not at all specific", 5: "Highly specific"},
)

q3 = QuestionMultipleChoice(
    question_name="skill_choice",
    question_text="What is the skill level required for this job: {{ job_post }}",
    question_options=["Entry level", "Intermediate", "Advanced", "Expert"],
)

q4 = QuestionYesNo(
    question_name="technical_yn",
    question_text="Is this a technical job? Job post: {{ job_post }}",
)

q5 = QuestionFreeText(
    question_name="rewrite_text",
    question_text="""Draft an improved version of the following job post: {{ job_post }}""",
)

## Building a survey
We combine the questions into a survey in order to administer them together:

In [3]:
from edsl import Survey

questions = [q1, q2, q3, q4, q5]

survey = Survey(questions)

If we want the agent/model to have information about prior questions in the survey we can add targeted or full memories ([learn more about adding survey rules/logic](https://docs.expectedparrot.com/en/latest/surveys.html)):

In [4]:
# Memory of a specific question is presented with another question:
# survey = survey.add_targeted_memory(q2, q1)

# Full memory of all prior questions is presented with each question (token-intensive):
# survey = survey.set_full_memory_mode()

## Adding data to the questions
We add the contents of each ticket into each question as an independent "scenario" for review. This allows us to create versions of the questions for each job post and deliver them to the model all at once. [EDSL provides many methods for generating scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) from different data sources (PDFs, CSVs, docs, images, tables, dicts, etc.). Here we import the list from above:

In [5]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList.from_list("job_post", job_posts)

## Designing AI agents
A key feature of EDSL is the ability to create personas for AI agents that the language models are prompted to use in generating responses to the questions. This is done by passing a dictionary of traits to Agent objects:

In [6]:
from edsl import AgentList, Agent

agent = Agent(traits={"persona":"You are a labor economist."})

## Selecting language models
EDSL allows us to select the language models to use in generating results. To see all available services:

In [7]:
from edsl import ModelList, Model

Model.services()

,Service Name
0,anthropic
1,azure
2,bedrock
3,deep_infra
4,deepseek
5,google
6,groq
7,mistral
8,ollama
9,openai


A current list of available models can be viewed [here](https://www.expectedparrot.com/getting-started/coop-pricing).

Here we select GPT 4o (if no model is specified, this model is also used by default):

In [8]:
model = Model("gpt-4o")

## Running the survey
We run the survey by adding the scenarios, agent and model with the `by()` method and then calling the `run()` method:

In [9]:
results = survey.by(scenarios).by(agent).by(model).run()

Job UUID,569cd72c-6973-46a6-a424-661d30c19c7f
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/569cd72c-6973-46a6-a424-661d30c19c7f
Exceptions Report URL,None
Results UUID,26700dc5-eb44-476b-bd5b-15de80da5873
Results URL,https://www.expectedparrot.com/content/26700dc5-eb44-476b-bd5b-15de80da5873


This generates a dataset of `Results` that we can analyze with built-in methods for data tables, dataframes, SQL, etc. We can see a list of all the components that can be analyzed:

In [10]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,agent.persona
4,answer.category_list
5,answer.rewrite_text
6,answer.skill_choice
7,answer.specific_scale
8,answer.technical_yn
9,cache_keys.category_list_cache_key


For example, we can filter, sort, select, limit, shuffle, sample and print some components of results in a table:

In [11]:
(
    results
    .filter("specific_scale in [3,4,5]")
    .sort_by("skill_choice")
    .select(
        "model",
        "persona",
        "job_post",
        "category_list",
        "specific_scale",
        "skill_choice",
        "technical_yn",
    )
)

,model.model,agent.persona,scenario.job_post,answer.category_list,answer.specific_scale,answer.skill_choice,answer.technical_yn
0,gpt-4o,You are a labor economist.,"Address customer inquiries and issues via phone and email, ensuring high levels of satisfaction and timely resolution.","['Customer Service', 'Customer Support Specialist', 'Phone and Email Support Representative']",3,Entry level,No
1,gpt-4o,You are a labor economist.,"Design creative visual content for marketing materials, including brochures, banners, and digital ads, using Adobe Creative Suite.","['Graphic Design', 'Marketing Graphic Design', 'Adobe Creative Suite Marketing Design']",5,Intermediate,Yes
2,gpt-4o,You are a labor economist.,"Assist veterinarians by preparing animals for surgery, administering injections, and providing post-operative care.","['Animal Care', 'Veterinary Support', 'Veterinary Surgical Assistant']",5,Intermediate,No
3,gpt-4o,You are a labor economist.,"Install and repair residential plumbing systems, including water heaters, pipes, and fixtures to ensure proper functionality.","['Plumbing Jobs', 'Residential Plumbing Jobs', 'Residential Plumbing Installation and Repair Jobs']",5,Intermediate,Yes
4,gpt-4o,You are a labor economist.,"Diagnose and repair automotive issues, perform routine maintenance, and ensure vehicles meet safety standards.","['Automotive Technician', 'Automotive Technician - Maintenance and Repair', 'Automotive Technician - Maintenance, Repair, and Safety Compliance']",3,Intermediate,Yes
5,gpt-4o,You are a labor economist.,Fold dough into pretzel shapes ensuring each is uniformly twisted and perfectly salted before baking.,"['Food Preparation', 'Baking', 'Pretzel Baking']",5,Intermediate,No


In [12]:
results.select("rewrite_text")

,answer.rewrite_text
0,"Job Title: Retail Operations Manager Job Description: We are seeking a dynamic and experienced Retail Operations Manager to lead and inspire our team in a vibrant and fast-paced retail environment. In this role, you will be responsible for overseeing daily operations, managing and developing staff, and ensuring an exceptional customer experience. Your leadership will be pivotal in driving efficiency, enhancing team performance, and achieving our business goals. Key Responsibilities: - Oversee and coordinate daily retail operations to ensure smooth and efficient functioning. - Lead, mentor, and develop a diverse team of staff to achieve their full potential. - Implement strategies to enhance customer satisfaction and ensure an outstanding shopping experience. - Monitor inventory levels and coordinate with suppliers to maintain optimal stock levels. - Analyze sales data to identify trends and develop strategies to improve sales performance. - Ensure compliance with company policies and industry regulations. - Foster a positive and inclusive work environment that encourages collaboration and innovation. Qualifications: - Proven experience in retail management or a similar role. - Strong leadership and team management skills. - Excellent communication and interpersonal abilities. - Ability to thrive in a fast-paced, dynamic environment. - Strong problem-solving skills and attention to detail. - Proficiency in retail management software and MS Office Suite. Join our team and play a key role in shaping the future of our retail operations. Apply today to make a meaningful impact in a company that values innovation and excellence."
1,"We are seeking a talented writer to create captivating and informative blog posts on health and wellness topics. Your work will play a crucial role in enhancing our website's traffic and engaging our readers. If you have a passion for health and wellness and a knack for crafting compelling content, we'd love to hear from you!"
2,"Job Title: Sales Data Analyst Job Description: We are seeking a skilled Sales Data Analyst to join our dynamic team. In this role, you will leverage your expertise in statistical analysis to examine sales data, identify key trends, and deliver actionable insights that will empower our marketing team to make informed decisions. Key Responsibilities: - Utilize advanced statistical tools and techniques to analyze sales data. - Identify patterns, trends, and correlations within the data to uncover valuable insights. - Collaborate with the marketing team to translate data findings into strategic recommendations. - Prepare comprehensive reports and visualizations to effectively communicate insights. - Continuously monitor data quality and implement improvements to enhance data accuracy. Qualifications: - Proven experience in data analysis, preferably in a sales or marketing environment. - Proficiency in statistical software and data visualization tools. - Strong analytical and problem-solving skills. - Excellent communication skills to convey complex data insights clearly and concisely. - Ability to work collaboratively with cross-functional teams. Join us and play a pivotal role in driving our marketing strategies through data-driven insights!"
3,"Join our culinary team as a Gourmet Chef, where your creativity and expertise will shine! We are seeking a talented individual to craft exquisite dishes that meet our high restaurant standards and captivate our guests with innovative and delightful flavor combinations. If you have a passion for culinary excellence and a flair for creating memorable dining experiences, we would love to hear from you."
4,"Join our dynamic team as a Creative Designer! We are seeking a talented individual to craft visually stunning marketing materials that captivate and engage. Your role will involve designing innovative brochures, eye-catching banners, and compelling digital ads using Adobe Creative Suite. If you have a keen eye for des

## Posting content to the Coop
We can post any objects to the Coop, including this notebook. Objects can be updated or modified at your Coop account, and shared with others or stored privately (default visibility is *unlisted*):

In [13]:
survey.push(description = "Example survey: Job posts analysis", visibility = "public")

{'description': 'Example survey: Job posts analysis',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/bbac6846-9857-46b4-9bf8-9508a068106d',
 'uuid': 'bbac6846-9857-46b4-9bf8-9508a068106d',
 'version': '0.1.45.dev1',
 'visibility': 'public'}

In [14]:
from edsl import Notebook

n = Notebook(path = "evaluating_job_posts.ipynb")

In [15]:
info = n.push(description = "Example text analysis: evaluating job posts", visibility = "public")
info

{'description': 'Example text analysis: evaluating job posts',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/91eb58b7-7740-4f12-8786-4424d0ed4bab',
 'uuid': '91eb58b7-7740-4f12-8786-4424d0ed4bab',
 'version': '0.1.45.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [16]:
n = Notebook(path = "evaluating_job_posts.ipynb") # resave the object

n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}